In [58]:
import xml.etree.ElementTree as ET
import pandas as pd
tree = ET.parse('NEWS2018_M-EnHi_trn.xml')
 
    
# getting the parent tag of
# the xml document
root = tree.getroot()
 

FileNotFoundError: [Errno 2] No such file or directory: 'NEWS2018_M-EnHi_trn.xml'

In [13]:
len(root)

12937

In [ ]:
tree = ET.parse('NEWS2018_M-EnHi_trn.xml')
 
# getting the parent tag of
# the xml document
root = tree.getroot()

english = []
hindi = []


for i in range(len(root)):
    english.append(root[i][0].text)
    hindi.append(root[i][1].text)

In [ ]:
df = pd.DataFrame({"english":english,"hindi":hindi})

In [ ]:
df.to_excel('train_data.xlsx',index=False)

In [52]:
tree = ET.parse('NEWS2018_M-EnHi_dev.xml')
 
# getting the parent tag of
# the xml document
root = tree.getroot()

english = []
hindi = []


for i in range(len(root)):
    english.append(root[i][0].text)
    hindi.append(root[i][1].text)

In [ ]:
df = pd.DataFrame({"english":english,"hindi":hindi})

In [ ]:
df.to_excel('test_data.xlsx',index=False)